In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import keras
import sklearn
import pickle

import sys
import logging 
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("py.warnings").setLevel(logging.ERROR)

%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 

In [ ]:
import general

In [ ]:
import simpsons

In [ ]:
config = dict(
    training_img_shape=(202,360),     # 1/4 of HD image
    test_img_shape=(202,360),         # ...
)

# Load model and thresholds

In [ ]:
tmp = pickle.load(open("data/model_and_thresholds.pickle", "rb"))
model, preprocess_pipeline, thresholds = tmp["model"], tmp["preprocess_pipeline"], tmp["thresholds"]

# Load datasets

In [ ]:
import gzip
import pickle

with gzip.GzipFile("data/dataset.pickle.zip", "r") as f:
    data = pickle.load(f)

print("train images by char: ", data.train.y.sum(axis=0))
print("train_dev images by char: ", data.train_dev.y.sum(axis=0))
print("dev  images by char: ", data.dev.y.sum(axis=0))
print("test images by char: ", data.test.y.sum(axis=0))
data.show()

# Score on testset

In [12]:
test_X = data.test.X
test_y = data.test.y
test_preds = model.predict(preprocess_pipeline.transform(test_X))
test_preds = (test_preds > thresholds).astype(int)

In [13]:
simpsons.scoring.final_report(test_y, test_preds)

Char #0 - bart
ACCURACY - 0.843
[[275   3]
 [ 52  21]]
             precision    recall  f1-score   support

        0.0       0.84      0.99      0.91       278
        1.0       0.88      0.29      0.43        73

avg / total       0.85      0.84      0.81       351




Char #1 - homer
ACCURACY - 0.758
[[214  35]
 [ 50  52]]
             precision    recall  f1-score   support

        0.0       0.81      0.86      0.83       249
        1.0       0.60      0.51      0.55       102

avg / total       0.75      0.76      0.75       351




Char #2 - lisa
ACCURACY - 0.895
[[283   4]
 [ 33  31]]
             precision    recall  f1-score   support

        0.0       0.90      0.99      0.94       287
        1.0       0.89      0.48      0.63        64

avg / total       0.89      0.89      0.88       351




Char #3 - marge
ACCURACY - 0.923
[[282   5]
 [ 22  42]]
             precision    recall  f1-score   support

        0.0       0.93      0.98      0.95       287
        1.0      

# Generate a Tagged Video

In [7]:
!cp data/preprocessing/test/full-episodes/The.Simpsons.S27E18.HDTV.x264-KILLERS[eztv].mkv   /tmp/video_file

In [8]:
tagger = simpsons.video.SimpsonsVideoTagger(model, thresholds, preprocess_pipeline=preprocess_pipeline)

In [9]:
clip = tagger.tag("/tmp/video_file", extractor_params=dict(num_frames=50))

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
clip.write_videofile("/tmp/video_tagged.mkv", fps=24)

In [ ]:
fr_ext = general.utils.VideoFrameExtractor("/tmp/video_file")
frames, frames_ts = fr_ext.extract(num_frames=50)
